## Классификация участников чата

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, SpatialDropout1D, Embedding, Conv1D, MaxPooling1D, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import utils

from google.colab import drive 

import os
import re
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


https://drive.google.com/file/d/1wk9jA6bJKIemzEnRZ74Xp61un6UZDPhp/view?usp=sharing - ссылка на файл

Набор данных представляет собой чат в мессенджере WhatsАpp за 2 года. В беседе участвует 7 человек

In [ ]:
#Загрузим беседу
all_texts=[]
for_german=[]
with open('Chat1.txt', encoding='utf-8') as text:
  vk = text.read()
vk = vk.split('\n')
for i in vk:
    text = i[20:]
    all_texts.append(text.split())

In [ ]:
# Чат имеет такой вид
vk[:10]

['\ufeff18.03.2019, 11:53 - Makka: Больше конференций богу конференций',
 '18.03.2019, 11:53 - Настя Егорова: Вот и добрались до Ромы с Герой)',
 '18.03.2019, 11:54 - Настя Дрянцева: Современные технологии?',
 '18.03.2019, 11:59 - Роман Куценко: Я сейчас как тру панк удалюсь из вотсапушки, и поставлю телеграмушку',
 '18.03.2019, 12:00 - Настя Дрянцева: Так мы тебя и там найдём',
 '18.03.2019, 12:00 - Настя Дрянцева: У большинства есть телеграм',
 '18.03.2019, 12:00 - Роман Куценко: Мне говорили там секретные чаты есть какие-то',
 '18.03.2019, 12:00 - Роман Куценко: Тогда ускоглазый wechat',
 '18.03.2019, 12:02 - Makka: Вайбiр православный!',
 '18.03.2019, 12:03 - Роман Куценко: Ахааааа']

In [ ]:
#Создадим на всякий случай словарь меток участников
name = ['Яна','Паша','Настя','Анастасия','Макс','Рома','Гера']
dictionary = {x:y for y,x in enumerate(name)}

In [ ]:
#Разделим набор данных по участникам
yana=[]
maks=[]
pasha=[]
nastia=[]
anastasia=[]
german=[]
roman=[]

for i in all_texts:
  if len(i) <2:
    continue
  if (i[0] == 'Настя') and (i[1] == 'Егорова:'):
    if len(i[2:]) >= 2:
      anastasia.append(i[2:])
  if i[0] == 'Makka:':
    if len(i[1:]) >= 2:
      maks.append(i[1:])
  if i[0] == 'ааПАШАаа:':
    if len(i[1:]) >= 2:
      pasha.append(i[1:])
  if i[0] == 'Енот:':
    if len(i[1:]) >= 2:    
      yana.append(i[1:])
  if i[0] == 'Роман':
    if len(i[2:]) >= 2:
      roman.append(i[2:])   
  if i[0] == 'Герман':
    if len(i[2:]) >= 2:
      german.append(i[2:])
  if (i[0] == 'Настя') and (i[1] == 'Дрянцева:'):
    if len(i[2:]) >= 2:
      nastia.append(i[2:])

In [ ]:
#Создадим наборы меток
label_yana=[0]*len(yana)
label_maks=[4]*len(maks)
label_pasha=[1]*len(pasha)
label_nastia=[2]*len(nastia)
label_anastasia=[3]*len(anastasia)
label_german=[6]*len(german)
label_roman=[5]*len(roman)

In [ ]:
#Соединим все сообщения и все метки
data = yana + maks + pasha + nastia + anastasia + german + roman
labels = label_yana + label_maks + label_pasha + label_nastia + label_anastasia + label_german + label_roman

In [ ]:
#Функция принимает на вход набор сообщений, метки к ним и желаемое количество слов в строке
def cut(data, labels, window):
  new_data = []
  step = len(data)//window
  start=0
  for i in range(step):
    new_data.append(data[start:start+window])
    start+=window
  new_data.append(data[-window:])
  new_labels = [labels]* len(new_data)
  return (new_data,new_labels) #Возвращает строки измененные по длине и соответствующие метки  

In [ ]:
#Проходим по набору сообщений, обрезаем длинные строки и добавляем лишнее в конец. Короткие строки заполняем до нужной длины
new_data=[]
new_labels =[]

for i,string in enumerate(data):
  while len(string) < 4:
    string.append('unknown')
  
  if len(string) > 4:
    cut_data, cut_labels = cut(string, labels[i], 4)
    new_data += cut_data
    new_labels += cut_labels
    data[i] = string[:4]


In [ ]:
#Собираем все данные в один набор
data += new_data
labels += new_labels

In [ ]:
#Кодируем метки с помощью OneHotEncoding
labels_hot = utils.to_categorical(labels, 7)

In [ ]:
#Разбиваем данные на обучающую и тестовую выборки
train_data, test_data, train_labels, test_labels = train_test_split(data, labels_hot, test_size=0.15, stratify=labels, shuffle=True)

In [ ]:
maxWordsCount = 11800 # Определяем максимальное количество слов/индексов, учитываемое при обучении текстов

tokenizer = Tokenizer(num_words=maxWordsCount, lower=False, oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(train_data) 
items = list(tokenizer.word_index.items()) 

In [ ]:
#Преобразуем строки в последовательность цифр
x_train = tokenizer.texts_to_sequences(train_data)
x_test = tokenizer.texts_to_sequences(test_data)

In [ ]:
print(len(x_train))
print(len(x_test))

9803
1730


In [ ]:
#Создаём сеть LSTM
model = Sequential()
model.add(Embedding(maxWordsCount, 20, input_length=4))
model.add(SpatialDropout1D(0.2))
model.add(BatchNormalization())
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(20, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=20,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/20
77/77 [==============================] - 4s 24ms/step - loss: 1.8993 - accuracy: 0.2259 - val_loss: 1.8562 - val_accuracy: 0.2705
Epoch 2/20
77/77 [==============================] - 1s 16ms/step - loss: 1.5233 - accuracy: 0.4335 - val_loss: 1.7507 - val_accuracy: 0.3058
Epoch 3/20
77/77 [==============================] - 1s 16ms/step - loss: 0.7944 - accuracy: 0.7445 - val_loss: 1.5716 - val_accuracy: 0.4087
Epoch 4/20
77/77 [==============================] - 1s 16ms/step - loss: 0.4169 - accuracy: 0.8703 - val_loss: 1.3216 - val_accuracy: 0.5422
Epoch 5/20
77/77 [==============================] - 1s 16ms/step - loss: 0.2508 - accuracy: 0.9220 - val_loss: 1.1456 - val_accuracy: 0.6116
Epoch 6/20
77/77 [==============================] - 1s 16ms/step - loss: 0.1947 - accuracy: 0.9364 - val_loss: 1.0831 - val_accuracy: 0.6457
Epoch 7/20
77/77 [==============================] - 1s 16ms/step - loss: 0.1668 - accuracy: 0.9442 - val_loss: 1.1409 - val_accuracy: 0.6497
Epoch 8/20
77

0.6601155996322632

In [ ]:
pred = [[50, 0.2, 'LSTM_50', 0.6584],
[50, 0.2, 'LSTM_100', 0.6635],
[20, 0.2, 'LSTM_100', 0.6682],
[20, 0.2, 'LSTM_150', 0.6578],
[20, 0.4, 'LSTM_100', 0.6635],
[20, 0.6, 'LSTM_100', 0.6607],
[50, 0.6, 'LSTM_100', 0.6682],
[20, 0.2, 'LSTM_100, LSTM_50', 0.6607],
[20, 0.2, 'LSTM_150, LSTM_100', 0.6624],
[50, 0.2, 'LSTM_150, LSTM_100', 0.6549],
[10, 0.2, 'LSTM_150, LSTM_100', 0.6555],
[20, 0.2, 'LSTM_50, LSTM_20, LSTM_10', 0.6624]]

pd.DataFrame(pred, columns = ['Embedding', 'SpatialDropout', 'Layers',  'Accuracy'])

,Embedding,SpatialDropout,Layers,Accuracy
0,50,0.2,LSTM_50,0.6584
1,50,0.2,LSTM_100,0.6635
2,20,0.2,LSTM_100,0.6682
3,20,0.2,LSTM_150,0.6578
4,20,0.4,LSTM_100,0.6635
5,20,0.6,LSTM_100,0.6607
6,50,0.6,LSTM_100,0.6682
7,20,0.2,"LSTM_100, LSTM_50",0.6607
8,20,0.2,"LSTM_150, LSTM_100",0.6624
9,50,0.2,"LSTM_150, LSTM_100",0.6549


Лучший результат для LSTM - 0.6682

In [ ]:
#Создаём сверточную сеть
model = Sequential()
model.add(Embedding(maxWordsCount, 50, input_length=4))
model.add(SpatialDropout1D(0.4))
model.add(BatchNormalization())
model.add(Conv1D(100, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Conv1D(50, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=50,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 3s 20ms/step - loss: 2.3729 - accuracy: 0.1634 - val_loss: 1.8851 - val_accuracy: 0.2705
Epoch 2/50
77/77 [==============================] - 1s 16ms/step - loss: 1.7307 - accuracy: 0.3352 - val_loss: 1.9223 - val_accuracy: 0.2705
Epoch 3/50
77/77 [==============================] - 2s 21ms/step - loss: 1.3924 - accuracy: 0.5032 - val_loss: 1.9447 - val_accuracy: 0.2705
Epoch 4/50
77/77 [==============================] - 1s 15ms/step - loss: 1.0850 - accuracy: 0.6337 - val_loss: 1.9136 - val_accuracy: 0.2792
Epoch 5/50
77/77 [==============================] - 1s 15ms/step - loss: 0.8023 - accuracy: 0.7349 - val_loss: 1.7370 - val_accuracy: 0.3543
Epoch 6/50
77/77 [==============================] - 1s 15ms/step - loss: 0.5984 - accuracy: 0.8107 - val_loss: 1.5492 - val_accuracy: 0.4133
Epoch 7/50
77/77 [==============================] - 1s 15ms/step - loss: 0.4553 - accuracy: 0.8561 - val_loss: 1.2783 - val_accuracy: 0.5786
Epoch 8/50
77

0.6589595079421997

In [ ]:
#Дообучим модель
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels.tolist(), epochs=1,batch_size=128, validation_data=(x_test, test_labels.tolist()))
np.max(history.history['val_accuracy'])

77/77 [==============================] - 3s 19ms/step - loss: 0.0860 - accuracy: 0.9654 - val_loss: 1.6261 - val_accuracy: 0.6538


0.6537572145462036

In [ ]:
pred = [[20, 0.2, 'Conv1D_50', 2, 0.6214],
[20, 0.2, 'Conv1D_100, Conv1D_50', 2, 0.6265],
[20, 0.2, 'Conv1D_100, Conv1D_50', (2,2), 0.6341],
[20, 0.2, 'Conv1D_200, Conv1D_100', (2,2), 0.6167],
[20, 0.2, 'Conv1D_50, Conv1D_20', (2,2), 0.6161],
[50, 0.2, 'Conv1D_100, Conv1D_50', (2,2), 0.6439],
[50, 0.2, 'Conv1D_100, Conv1D_100', (2,2), 0.6375],
[50, 0.4, 'Conv1D_100, Conv1D_50', (2,2), 0.6632],
[50, 0.6, 'Conv1D_100, Conv1D_50', (2,2), 0.6635],
[50, 0.8, 'Conv1D_100, Conv1D_50', (2,2), 0.6558],
[100, 0.4, 'Conv1D_100, Conv1D_50', (2,2), 0.6549],
[100, 0.4, 'Conv1D_150, Conv1D_100', (2,2), 0.6578],
[100, 0.4, 'Conv1D_50, Conv1D_100', (2,2), 0.6491]]

pd.DataFrame(pred, columns = ['Embedding', 'SpatialDropout', 'Layers', 'MaxPooling', 'Accuracy'])

,Embedding,SpatialDropout,Layers,MaxPooling,Accuracy
0,20,0.2,Conv1D_50,2,0.6214
1,20,0.2,"Conv1D_100, Conv1D_50",2,0.6265
2,20,0.2,"Conv1D_100, Conv1D_50","(2, 2)",0.6341
3,20,0.2,"Conv1D_200, Conv1D_100","(2, 2)",0.6167
4,20,0.2,"Conv1D_50, Conv1D_20","(2, 2)",0.6161
5,50,0.2,"Conv1D_100, Conv1D_50","(2, 2)",0.6439
6,50,0.2,"Conv1D_100, Conv1D_100","(2, 2)",0.6375
7,50,0.4,"Conv1D_100, Conv1D_50","(2, 2)",0.6632
8,50,0.6,"Conv1D_100, Conv1D_50","(2, 2)",0.6635
9,50,0.8,"Conv1D_100, Conv1D_50","(2, 2)",0.6558


Лучший результат для сверточной сети - 0.6635

Попробуем варианты LSTM+Dense

In [ ]:
#Создаём полносвязную сеть
model = Sequential()
# model.add(Embedding(maxWordsCount, 50, input_length=4))
# model.add(SpatialDropout1D(0.4))
# model.add(BatchNormalization())
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(20, return_sequences=True))
# model.add(LSTM(10))
# model = Sequential()
model.add(Embedding(maxWordsCount, 50, input_length=4))
model.add(SpatialDropout1D(0.4))
model.add(BatchNormalization())
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(20, return_sequences=True))
model.add(LSTM(10))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(100, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(100, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
# model.add(Dense(200, activation='relu'))
# model.add(Dropout(0.1))
# model.add(BatchNormalization())
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.1))
# model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=50,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 9s 40ms/step - loss: 2.4690 - accuracy: 0.1508 - val_loss: 1.9497 - val_accuracy: 0.2705
Epoch 2/50
77/77 [==============================] - 2s 23ms/step - loss: 1.9699 - accuracy: 0.2673 - val_loss: 1.8849 - val_accuracy: 0.2705
Epoch 3/50
77/77 [==============================] - 2s 22ms/step - loss: 1.5927 - accuracy: 0.4268 - val_loss: 1.8462 - val_accuracy: 0.2705
Epoch 4/50
77/77 [==============================] - 2s 23ms/step - loss: 1.1334 - accuracy: 0.6107 - val_loss: 1.7613 - val_accuracy: 0.2919
Epoch 5/50
77/77 [==============================] - 2s 22ms/step - loss: 0.8009 - accuracy: 0.7452 - val_loss: 1.5220 - val_accuracy: 0.4121
Epoch 6/50
77/77 [==============================] - 2s 22ms/step - loss: 0.5704 - accuracy: 0.8179 - val_loss: 1.2630 - val_accuracy: 0.5665
Epoch 7/50
77/77 [==============================] - 2s 23ms/step - loss: 0.4325 - accuracy: 0.8708 - val_loss: 1.2895 - val_accuracy: 0.5948
Epoch 8/50
77

0.6589595079421997

Попробуем варианты LSTM+Conv1D

In [ ]:
#Создаём сверточную сеть
model = Sequential()
model.add(Embedding(maxWordsCount, 50, input_length=4))
model.add(SpatialDropout1D(0.4))
model.add(BatchNormalization())
model.add(Conv1D(100, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Conv1D(50, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=50,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 3s 20ms/step - loss: 2.3842 - accuracy: 0.1620 - val_loss: 1.9338 - val_accuracy: 0.1757
Epoch 2/50
77/77 [==============================] - 1s 16ms/step - loss: 1.9620 - accuracy: 0.2529 - val_loss: 1.8685 - val_accuracy: 0.2705
Epoch 3/50
77/77 [==============================] - 1s 16ms/step - loss: 1.6519 - accuracy: 0.3893 - val_loss: 1.8358 - val_accuracy: 0.3064
Epoch 4/50
77/77 [==============================] - 1s 16ms/step - loss: 1.2216 - accuracy: 0.5648 - val_loss: 1.7722 - val_accuracy: 0.3081
Epoch 5/50
77/77 [==============================] - 1s 16ms/step - loss: 0.8338 - accuracy: 0.7146 - val_loss: 1.5687 - val_accuracy: 0.3867
Epoch 6/50
77/77 [==============================] - 1s 16ms/step - loss: 0.5928 - accuracy: 0.7989 - val_loss: 1.3904 - val_accuracy: 0.5023
Epoch 7/50
77/77 [==============================] - 1s 16ms/step - loss: 0.4387 - accuracy: 0.8547 - val_loss: 1.2386 - val_accuracy: 0.5873
Epoch 8/50
77

0.6583815217018127

Лучшего результата добиться не получилось

Посмотрим каких участников чата было сложнее всего правильно определить

In [ ]:
#Запишем предсказания модели на тестовой выборке и преобразуем их вид
prediction = model.predict(x_test)
y_pred = [np.argmax(x) for x in prediction]
test_l = [np.argmax(x) for x in test_labels]

In [ ]:
#Разделим предсказаные метки на правильные и неправильные
right = []
wrong = []

for i in range(len(y_pred)):
  if y_pred[i] == test_l[i]:
    right.append(y_pred[i])
  else:
    wrong.append(y_pred[i])

In [ ]:
#Посчитаем долю правильных меток для каждого участника чата
right_array = np.array(right)
wrong_array = np.array(wrong)
r =np.zeros(7)
w= np.zeros(7)

for i in range(7):
    r[i] = len(right_array[right_array==i])
    w[i] = len(wrong_array[wrong_array==i])

all = r + w
procent = r/all

In [ ]:
name = ['Яна','Паша','Настя','Анастасия','Макс','Рома','Гера']

In [ ]:
for i in range(7):
  print('{} {}% сеть определила правильно'.format(name[i],round(procent[i]*100)))

Яна 56% сеть определила правильно
Паша 67% сеть определила правильно
Настя 71% сеть определила правильно
Анастасия 74% сеть определила правильно
Макс 51% сеть определила правильно
Рома 69% сеть определила правильно
Гера 80% сеть определила правильно


Попробуем убрать 15 самых часто употребляемых слов из в наборе

In [ ]:
maxWordsCount = 11800 # Определяем максимальное количество слов/индексов, учитываемое при обучении текстов

tokenizer = Tokenizer(num_words=maxWordsCount, lower=False, oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(train_data) 
items = list(tokenizer.word_index.keys()) 

In [ ]:
items = items[:15]

In [ ]:
#Занулим первые 15 слов
for i in items:
  tokenizer.word_index[i] = 0

In [ ]:
x_train = tokenizer.texts_to_sequences(train_data)
x_test = tokenizer.texts_to_sequences(test_data)

In [ ]:
#Рекуррентная сеть
model = Sequential()
model.add(Embedding(maxWordsCount, 50, input_length=4))
model.add(SpatialDropout1D(0.4))
model.add(BatchNormalization())
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(20, return_sequences=True))
model.add(LSTM(10))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=50,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 8s 37ms/step - loss: 1.9149 - accuracy: 0.2348 - val_loss: 1.8687 - val_accuracy: 0.2705
Epoch 2/50
77/77 [==============================] - 2s 20ms/step - loss: 1.6993 - accuracy: 0.3459 - val_loss: 1.7934 - val_accuracy: 0.3746
Epoch 3/50
77/77 [==============================] - 2s 20ms/step - loss: 1.3093 - accuracy: 0.5342 - val_loss: 1.6150 - val_accuracy: 0.4116
Epoch 4/50
77/77 [==============================] - 2s 20ms/step - loss: 0.9633 - accuracy: 0.6634 - val_loss: 1.4486 - val_accuracy: 0.4717
Epoch 5/50
77/77 [==============================] - 2s 20ms/step - loss: 0.6819 - accuracy: 0.8035 - val_loss: 1.3323 - val_accuracy: 0.5520
Epoch 6/50
77/77 [==============================] - 2s 21ms/step - loss: 0.5280 - accuracy: 0.8483 - val_loss: 1.2364 - val_accuracy: 0.5977
Epoch 7/50
77/77 [==============================] - 2s 21ms/step - loss: 0.3881 - accuracy: 0.8933 - val_loss: 1.2496 - val_accuracy: 0.6173
Epoch 8/50
77

0.6612716913223267

In [ ]:
#Сверточная сеть
model = Sequential()
model.add(Embedding(maxWordsCount, 50, input_length=4))
model.add(SpatialDropout1D(0.4))
model.add(BatchNormalization())
model.add(Conv1D(100, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Conv1D(50, 2, activation='relu',padding='same'))
model.add(MaxPooling1D(2))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, train_labels.tolist(), epochs=50,batch_size=128, validation_data=(x_test, test_labels.tolist()))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 3s 19ms/step - loss: 2.3492 - accuracy: 0.1715 - val_loss: 1.8727 - val_accuracy: 0.2705
Epoch 2/50
77/77 [==============================] - 1s 15ms/step - loss: 1.7272 - accuracy: 0.3339 - val_loss: 1.8866 - val_accuracy: 0.2705
Epoch 3/50
77/77 [==============================] - 1s 15ms/step - loss: 1.4029 - accuracy: 0.5049 - val_loss: 1.8855 - val_accuracy: 0.2705
Epoch 4/50
77/77 [==============================] - 1s 15ms/step - loss: 1.0912 - accuracy: 0.6361 - val_loss: 1.8631 - val_accuracy: 0.2705
Epoch 5/50
77/77 [==============================] - 1s 15ms/step - loss: 0.8248 - accuracy: 0.7278 - val_loss: 1.7475 - val_accuracy: 0.3145
Epoch 6/50
77/77 [==============================] - 1s 15ms/step - loss: 0.5852 - accuracy: 0.8211 - val_loss: 1.5342 - val_accuracy: 0.4168
Epoch 7/50
77/77 [==============================] - 1s 15ms/step - loss: 0.4409 - accuracy: 0.8587 - val_loss: 1.2389 - val_accuracy: 0.5711
Epoch 8/50
77

0.6531791687011719

Лучего результата добиться не получилось

Обученим модель с использование BagOfWords.

In [ ]:
xTrain01 = tokenizer.texts_to_matrix(train_data)
xTest01 = tokenizer.texts_to_matrix(test_data)

In [ ]:
#Создаём полносвязную сеть
model = Sequential()
model.add(Dense(300, input_dim = 11800, activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(200, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(100, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(xTrain01, np.array(train_labels), epochs=20, batch_size=128, validation_data=(xTest01, np.array(test_labels)))

np.max(history.history['val_accuracy'])

Epoch 1/50
77/77 [==============================] - 6s 63ms/step - loss: 2.2855 - accuracy: 0.2181 - val_loss: 1.9208 - val_accuracy: 0.1734
Epoch 2/50
77/77 [==============================] - 5s 59ms/step - loss: 0.7119 - accuracy: 0.7653 - val_loss: 1.7692 - val_accuracy: 0.3046
Epoch 3/50
77/77 [==============================] - 5s 59ms/step - loss: 0.1865 - accuracy: 0.9487 - val_loss: 1.5366 - val_accuracy: 0.4121
Epoch 4/50
77/77 [==============================] - 5s 59ms/step - loss: 0.1161 - accuracy: 0.9674 - val_loss: 1.2423 - val_accuracy: 0.6156
Epoch 5/50
77/77 [==============================] - 5s 67ms/step - loss: 0.0895 - accuracy: 0.9715 - val_loss: 1.0144 - val_accuracy: 0.6636
Epoch 6/50
77/77 [==============================] - 5s 59ms/step - loss: 0.0792 - accuracy: 0.9731 - val_loss: 0.9595 - val_accuracy: 0.6792
Epoch 7/50
77/77 [==============================] - 5s 60ms/step - loss: 0.0741 - accuracy: 0.9725 - val_loss: 1.0437 - val_accuracy: 0.6775
Epoch 8/50
77

KeyboardInterrupt: ignored

In [ ]:
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(xTrain01, np.array(train_labels), epochs=10, batch_size=128, validation_data=(xTest01, np.array(test_labels)))
np.max(history.history['val_accuracy'])

Epoch 1/10
77/77 [==============================] - 6s 62ms/step - loss: 0.0593 - accuracy: 0.9746 - val_loss: 1.6002 - val_accuracy: 0.6751
Epoch 2/10
77/77 [==============================] - 5s 63ms/step - loss: 0.0577 - accuracy: 0.9751 - val_loss: 1.5994 - val_accuracy: 0.6751
Epoch 3/10
77/77 [==============================] - 5s 60ms/step - loss: 0.0584 - accuracy: 0.9749 - val_loss: 1.6037 - val_accuracy: 0.6757
Epoch 4/10
77/77 [==============================] - 5s 65ms/step - loss: 0.0632 - accuracy: 0.9744 - val_loss: 1.6035 - val_accuracy: 0.6769
Epoch 5/10
77/77 [==============================] - 5s 60ms/step - loss: 0.0607 - accuracy: 0.9760 - val_loss: 1.6046 - val_accuracy: 0.6757
Epoch 6/10
77/77 [==============================] - 5s 60ms/step - loss: 0.0654 - accuracy: 0.9738 - val_loss: 1.6081 - val_accuracy: 0.6746
Epoch 7/10
77/77 [==============================] - 5s 59ms/step - loss: 0.0575 - accuracy: 0.9754 - val_loss: 1.6047 - val_accuracy: 0.6728
Epoch 8/10
77

0.6768786311149597

Лучший результат для BagOfWords - 0.6792

Вывод: Полносвязная модель с BagOfWords показала результат лучше чем CNN или RNN. Возможно причина в том, что сообщения очень короткие и смысла в запоминании последовательности слов особо нет. Может быть мне не удалось подобрать наилучшие комбинации параметров для CNN/RNN.